In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
import shapely
import ee
import geemap

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-coberturap')

In [4]:
# --- File paths  ---
root_folder=r"/notebooks/ProyectoFinal/Shapes/"
adi_route = root_folder+'/PNNAmacayacu.shp'
adi = gpd.read_file(adi_route).to_crs(epsg=9377)
adi

,objectid_1,nombre,categoria,territoria,resolucion,hectareas_,escala,organizaci,fecha_act,url,...,centroid_y,fecha_regi,validado,hectarea_1,perimetro_,administra,id_pnn,record_id,app_id,geometry
0,198761,Amacayacu,Parque Nacional Natural,Territorial Amazonia,283,267480.32,None,Parques Nacionales Naturales de Colombia,2011-06-30,https://runap.parquesnacionales.gov.co/area-pr...,...,-3.439523,2011-06-30,Si,267240.603115,None,PNNC,27,2069,1,"POLYGON ((5331866.854 1221528.042, 5331899.919..."


In [5]:
ee_fc = geemap.gdf_to_ee(adi)     # Esto convierte todo el shapefile a una EE FeatureCollection
adi = ee_fc.geometry()

In [6]:
dataset = (ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')
           .filterDate('2022-10-01', '2023-12-31')
           .filterBounds(adi))

info = dataset.getInfo()
print('Número de imágenes:', len(info['features']))


Número de imágenes: 6


In [9]:
##Function to select highest quality GEDI data
def quality_mask(image):
    return image.updateMask(image.select('l4_quality_flag').eq(1)) \
                .updateMask(image.select('degrade_flag').eq(0))

 
##Function to mask unreliable GEDI measurements
## with a relative standard error > 50% 
## agbd_se / agbd > 0.5
def errorMask(image):
  relative_se = image.select('agbd_se').divide(image.select('agbd'))
  return image.updateMask(relative_se.lte(0.6));


gediProjection = ee.Image(dataset.first()).select('agbd').projection()
gediProcessed = dataset.map(quality_mask).map(errorMask)
gediMosaic = gediProcessed.mosaic().select('agbd').setDefaultProjection(gediProjection);

##Definir los parámetros de visualización
agbdVis = {
  min: 0,
  max: 300,  ##Puedes ajustar según el rango de tus datos
  'palette': ['f7fcf5', 'c7e9c0', '74c476', '31a354', '006d2c', '00441b']
};

Map = geemap.Map()
Map.centerObject(adi)
Map.addLayer(gediMosaic, agbdVis, 'GEDI AGBD', True, 1)
Map.addLayer(adi, {}, 'PNN Amacayacu', 0)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…